In [1]:
%pip install open_clip_torch

Note: you may need to restart the kernel to use updated packages.


In [5]:
import torch
from PIL import Image
import open_clip


#  ('coca_ViT-B-32', 'laion2b_s13b_b90k'),
#  ('coca_ViT-B-32', 'mscoco_finetuned_laion2b_s13b_b90k'),
#  ('coca_ViT-L-14', 'laion2b_s13b_b90k'),
#  ('coca_ViT-L-14', 'mscoco_finetuned_laion2b_s13b_b90k'),


model, _, preprocess = open_clip.create_model_and_transforms('coca_ViT-B-32', pretrained='laion2b_s13b_b90k')
model.eval()  # model in train mode by default, impacts some models with BatchNorm or stochastic depth active
tokenizer = open_clip.get_tokenizer('coca_ViT-B-32')

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
import requests
from PIL import Image
image = Image.open(requests.get(url, stream=True).raw)
import matplotlib.pyplot as plt

image = preprocess(image).unsqueeze(0)
args = ["a photo of a cat", "a photo of a dog", "two remote controls"]
text = tokenizer(args)

with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

print("Label probs:", text_probs)  # prints: [[1., 0., 0.]]
for i, arg in enumerate(args):
    prob = text_probs[0, i].item()
    print(f"{arg:<20} {prob * 100:.2f}%")

open_clip_pytorch_model.bin:   0%|          | 0.00/1.01G [00:00<?, ?B/s]